# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.113005e+02     1.695879e+00
 * time: 0.08987212181091309
     1     1.062714e+01     8.704111e-01
 * time: 0.9242010116577148
     2    -1.215558e+01     1.055267e+00
 * time: 1.1894850730895996
     3    -3.413548e+01     7.939458e-01
 * time: 1.3698070049285889
     4    -4.763283e+01     5.693305e-01
 * time: 1.7670519351959229
     5    -5.707730e+01     2.078044e-01
 * time: 1.922076940536499
     6    -5.992332e+01     1.389056e-01
 * time: 2.039149045944214
     7    -6.097523e+01     4.757450e-02
 * time: 2.156002998352051
     8    -6.136758e+01     5.994065e-02
 * time: 2.266170024871826
     9    -6.159949e+01     3.258536e-02
 * time: 2.383674144744873
    10    -6.180025e+01     3.503339e-02
 * time: 2.5003421306610107
    11    -6.196544e+01     3.049756e-02
 * time: 2.617309093475342
    12    -6.204564e+01     2.541724e-02
 * time: 2.732762098312378
    13    -6.212381e+01     1.534821e-02
 * time: 2.8421130180358887

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671060
    AtomicLocal         -18.8557678
    AtomicNonlocal      14.8522657
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485366 
    Xc                  -19.3336818

    total               -62.261666460929
